<a href="https://colab.research.google.com/github/debashisdotchatterjee/dax_ti_bayes_property_YM/blob/main/dax_ti_bayes_property_YM_Data_Code_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Important upfront note

This notebook implements a **hierarchical Bayesian regression per property**, featuring:

- **Latent true values** \( z_i \) to handle measurement error and missing data  
- **Class-specific regression coefficients** \( \beta_g \) for each `moe_class`  
- A **global coefficient mean** \( \mu_\beta \) for partial pooling across classes  
- **Inverse-gamma priors** on:
  - Residual variance \( \sigma^2 \)  
  - Between-class variance \( \tau^2 \)  
- A **full Gibbs sampler** written by hand (no MCMC libraries)

---

###  Modeling simplification

To keep the code tractable and scriptable:

- Each mechanical property is modeled **independently**  
- This results in **parallel univariate hierarchical models** using a common design matrix \( X \)  
- This is a simplification of the fully multivariate \( \Sigma \)-coupled model described in the LaTeX paper  
- The structure and logic are preserved, but \( \Sigma \) is treated as **diagonal**, i.e., one \( \sigma_k^2 \) per property

> Extending this to a full **matrix-normal multi-output model** is algebraically straightforward but computationally intensive.  
> This code can serve as the **practical implementation section** of your paper, clearly noting it uses the “diagonal \( \Sigma \)” version.

---

### ▶ What this Colab cell does

1. **Load** `dax-ti-static.csv`
2. **Build a design matrix** \( X \) from:
   - Intercept  
   - Standardised `moe`, `P0` (oxygen), log-transformed grain size `P1`  
   - One-hot dummies for `moe_class`, `condition`, `ph1`
3. **For each property** (`YM`, `YS`, `UTS`, `DAR`, `HV`):
   - Run manual Gibbs MCMC with a `tqdm` progress bar  
   - Save:
     - Trace plots  
     - Posterior histograms  
     - Predictive plots  
     - Residual plots  
   - Save summary tables  
4. **Create a ZIP** of the entire results folder



## Corrections 1: Making the Sampling Numerically Robust

To ensure stable and reproducible inference, the following adjustments are made:

- **Sampling parameters**:
  - `N_ITER = 10000`  
  - `BURN_IN = 4000`  
  - `THIN = 10`

- **Custom inverse-gamma sampler**:
  - Uses NumPy’s `np.random.gamma` instead of `scipy.stats.invgamma.rvs`  
  - Includes **strict checks** on shape and scale parameters to prevent instability

- **NaN-safe computation**:
  - Uses `np.nansum` to prevent NaNs from propagating into shape/scale calculations

---

###  All other logic remains unchanged

- Hierarchical modeling **per property**  
- Latent variables \( z_i \)  
- Class-specific coefficients \( \beta_g \)  
- Global mean \( \mu_\beta \)  
- Manual Gibbs sampler with `tqdm` progress bars  
- Posterior plots, predictive diagnostics, residuals  
- Summary tables and ZIP archive of results

---

This setup ensures robust sampling while preserving the full modeling structure.

In [ ]:
"""
Hierarchical Bayesian surrogate for DAX-Ti dataset
==================================================

- Manual Gibbs sampler (no PyMC/Stan/etc.)
- One hierarchical model *per property* (YM, YS, UTS, DAR, HV)
- Each property uses:
    z_i      ~ N(x_i^T beta_{g_i}, sigma^2)
    y_i_obs  ~ N(z_i, s_i^2)  (measurement error)
    beta_g   ~ N(mu_beta, tau^2 I_p)
    mu_beta  ~ N(0, c0^2 I_p)
    sigma^2  ~ Inv-Gamma(a0, b0)
    tau^2    ~ Inv-Gamma(a_tau, b_tau)

- g_i = Mo-equivalent stability class index from 'moe_class' column.

Outputs:
--------
- All figures and tables saved in `results_dax_ti_bayes/`
- A zip file `results_dax_ti_bayes.zip` containing everything
- Key summaries printed to stdout.

Run in Colab:
-------------
1. Upload `dax-ti-static.csv` to your working directory.
2. Paste this whole script in a cell and run.
"""

import os
import zipfile
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

# =========================================================
# Configuration
# =========================================================

CSV_PATH = "dax-ti-static.csv"   # adjust if needed
OUTPUT_DIR = "results_dax_ti_bayes"

# MCMC settings
N_ITER = 10000       # total iterations
BURN_IN = 6000       # burn-in
THIN = 10            # thinning factor
RANDOM_SEED = 2025

np.random.seed(RANDOM_SEED)

# global placeholder for feature names used inside functions
feature_names = None


# =========================================================
# Utility functions
# =========================================================

def ensure_dir(path):
    Path(path).mkdir(parents=True, exist_ok=True)


def standardize_series(s):
    """Return standardized series and (mean, std)."""
    mu = s.mean()
    sd = s.std(ddof=1)
    if sd == 0 or np.isnan(sd):
        sd = 1.0
    return (s - mu) / sd, mu, sd


def invgamma_sample(a, b):
    """
    Sample from Inv-Gamma(a, b) using 1 / Gamma(shape=a, scale=1/b).
    Guards against invalid shape/scale.
    """
    a = float(a)
    b = float(b)
    if not np.isfinite(a) or a <= 0:
        a = 1.0
    if not np.isfinite(b) or b <= 0:
        b = 1.0
    g = np.random.gamma(shape=a, scale=1.0 / b)
    if g <= 0 or not np.isfinite(g):
        g = 1.0
    return 1.0 / g


def create_zip_from_directory(directory, zip_name):
    with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(directory):
            for f in files:
                full_path = os.path.join(root, f)
                rel_path = os.path.relpath(full_path, directory)
                zf.write(full_path, arcname=rel_path)


def safe_sample_mvn(mean, cov, jitter=1e-8, max_tries=5):
    """
    Numerically robust sampler from N(mean, cov) using eigen-decomposition and jitter.
    Avoids SVD non-convergence issues from np.random.multivariate_normal.
    """
    p = cov.shape[0]
    cov = 0.5 * (cov + cov.T)  # enforce symmetry

    for _ in range(max_tries):
        try:
            w, v = np.linalg.eigh(cov)
            w = np.clip(w, jitter, None)
            L = v @ np.diag(np.sqrt(w))
            z = np.random.randn(p)
            return mean + L @ z
        except np.linalg.LinAlgError:
            cov = cov + jitter * np.eye(p)
            jitter *= 10.0

    # last resort: diagonal approx
    diag_cov = np.diag(np.clip(np.diag(cov), jitter, None))
    z = np.random.randn(p)
    L = np.sqrt(np.diag(diag_cov))
    return mean + L * z


def compute_mu_reg(X, beta_g, groups):
    """
    Compute mu_reg[i] = x_i^T beta_{g_i} for all i.
    X: (n x p), beta_g: (J x p), groups: (n,)
    """
    XB = np.einsum("ij,jk->ik", X, beta_g.T)
    idx = np.arange(X.shape[0])
    return XB[idx, groups]


# =========================================================
# Data preparation
# =========================================================

def load_and_prepare_data(csv_path):
    """
    Load dax-ti-static.csv and construct:
    - df: cleaned DataFrame
    - X: design matrix (n x p)
    - feature_names: list of feature names
    - groups: integer moe_class indices (0..J-1)
    - group_labels: mapping index -> original string
    - y_dict: dict of observed values per property (with NaN for missing)
    - s_dict: dict of measurement std dev per property
    """
    df = pd.read_csv(csv_path)

    # Basic sanity: remove spaces in column names (just in case)
    df.columns = [c.strip() for c in df.columns]

    # Filter to rows that have moe_class; drop if missing
    df = df[~df["moe_class"].isna()].copy()

    # Make sure required columns exist
    required_cols = [
        "moe", "moe_class", "P0", "P1",
        "YM", "YM_err",
        "YS", "YS_err",
        "UTS", "UTS_err",
        "DAR", "DAR_err",
        "HV", "HV_err",
        "condition",
        "ph1"
    ]
    for c in required_cols:
        if c not in df.columns:
            raise ValueError(f"Column '{c}' not found in CSV. Please check the dataset.")

    # Handle grain size: log-transform (P1 can be zero / missing)
    df["P1_log"] = np.log1p(df["P1"].fillna(df["P1"].median()))

    # Standardize numeric covariates
    covariates_cont = ["moe", "P0", "P1_log"]
    for c in covariates_cont:
        df[c + "_std"], _, _ = standardize_series(df[c])

    # One-hot encode categorical covariates: condition, ph1
    cat_cols = ["condition", "ph1"]
    df[cat_cols] = df[cat_cols].fillna("missing")
    df_dummies = pd.get_dummies(df[cat_cols], prefix=cat_cols, drop_first=True)

    # Build design matrix X
    X_df = pd.concat(
        [
            pd.Series(1.0, index=df.index, name="Intercept"),
            df[[c + "_std" for c in covariates_cont]],
            df_dummies
        ],
        axis=1
    )
    feature_names_local = list(X_df.columns)
    X = X_df.to_numpy(dtype=float)

    # Map moe_class to integer groups
    moe_classes = df["moe_class"].astype(str)
    class_levels = sorted(moe_classes.unique())
    class_to_idx = {c: i for i, c in enumerate(class_levels)}
    idx_to_class = {i: c for c, i in class_to_idx.items()}
    groups = moe_classes.map(class_to_idx).to_numpy(dtype=int)

    # Prepare observed y and s (measurement std dev) per property
    prop_cols = {
        "YM": ["YM", "YM_err"],
        "YS": ["YS", "YS_err"],
        "UTS": ["UTS", "UTS_err"],
        "DAR": ["DAR", "DAR_err"],
        "HV": ["HV", "HV_err"],
    }

    y_dict = {}
    s_dict = {}

    for prop_name, (v_col, e_col) in prop_cols.items():
        y = df[v_col].to_numpy(dtype=float)
        s = df[e_col].to_numpy(dtype=float)

        # If error is missing or zero, set a default relative error
        rel_default = 0.05  # 5% relative if err missing
        s_clean = s.copy()
        mask_bad = np.isnan(s_clean) | (s_clean <= 0)
        s_clean[mask_bad] = rel_default * np.abs(y[mask_bad])

        y_dict[prop_name] = y
        s_dict[prop_name] = s_clean

    return df, X, feature_names_local, groups, idx_to_class, y_dict, s_dict


# =========================================================
# Transformations (log scale etc.)
# =========================================================

def transform_property(y, s, prop_name):
    """
    Apply transformations:
    - log for YM, YS, UTS, HV
    - log for DAR, but only for positive tensiles (DAR > 0). Others treated as missing.
    Returns:
    - z_obs: transformed observations (NaN if missing/invalid)
    - s_trans: approx transformed std dev (delta method)
    """
    y = y.copy()
    s = s.copy()

    if prop_name in ["YM", "YS", "UTS", "HV"]:
        invalid = (y <= 0) | np.isnan(y)
        z_obs = np.full_like(y, np.nan, dtype=float)
        z_obs[~invalid] = np.log(y[~invalid])

        s_trans = np.full_like(s, np.nan, dtype=float)
        s_trans[~invalid] = np.abs(s[~invalid] / y[~invalid])

    elif prop_name == "DAR":
        invalid = (y <= 0) | np.isnan(y)
        z_obs = np.full_like(y, np.nan, dtype=float)
        z_obs[~invalid] = np.log(y[~invalid])

        s_trans = np.full_like(s, np.nan, dtype=float)
        s_trans[~invalid] = np.abs(s[~invalid] / y[~invalid])

    else:
        raise ValueError(f"Unknown property name: {prop_name}")

    return z_obs, s_trans


# =========================================================
# MCMC for one property
# =========================================================

def run_hierarchical_mcmc_property(
    X,
    groups,
    z_obs,
    s_obs,
    prop_name,
    idx_to_class,
    n_iter=N_ITER,
    burn_in=BURN_IN,
    thin=THIN,
    output_dir=OUTPUT_DIR
):
    """
    Run Gibbs sampler for one property.

    Model:
      z_i | beta_{g_i}, sigma^2 ~ N(x_i^T beta_{g_i}, sigma^2)
      z_obs_i | z_i, s_i^2 ~ N(z_i, s_i^2)  (if observed)
      beta_g | mu_beta, tau^2 ~ N(mu_beta, tau^2 I_p)
      mu_beta ~ N(0, c0^2 I_p)
      sigma^2 ~ Inv-Gamma(a0, b0)
      tau^2   ~ Inv-Gamma(a_tau, b_tau)
    """
    ensure_dir(output_dir)
    prop_dir = os.path.join(output_dir, f"property_{prop_name}")
    ensure_dir(prop_dir)

    n, p = X.shape
    J = len(np.unique(groups))

    # Masks
    obs_mask = ~np.isnan(z_obs)

    # Replace NaN s_obs with a large value (if any remain)
    s_obs_clean = s_obs.copy()
    s_obs_clean[np.isnan(s_obs_clean)] = 1e6

    # Hyperparameters
    a0, b0 = 2.0, 1.0          # prior for sigma^2
    a_tau, b_tau = 2.0, 1.0    # prior for tau^2
    c0 = 10.0                  # prior scale for mu_beta
    jitter_prec = 1e-6         # ridge for precision matrix

    # Initialize parameters
    z = np.zeros(n)
    z[obs_mask] = z_obs[obs_mask]
    z[~obs_mask] = 0.0

    beta_g = np.zeros((J, p))
    mu_beta = np.zeros(p)
    sigma2 = 1.0
    tau2 = 1.0

    # Precompute index sets per group
    group_indices = {g: np.where(groups == g)[0] for g in range(J)}

    # Storage
    n_keep = (n_iter - burn_in) // thin
    beta_samples = np.zeros((n_keep, J, p))
    mu_beta_samples = np.zeros((n_keep, p))
    sigma2_samples = np.zeros(n_keep)
    tau2_samples = np.zeros(n_keep)

    # Gibbs sampler
    print(f"\n=== MCMC for property: {prop_name} ===")
    print(f"n = {n}, p = {p}, J (moe_class) = {J}")
    print(f"Saving results to: {prop_dir}")

    keep_idx = 0

    for it in tqdm(range(1, n_iter + 1), desc=f"MCMC {prop_name}"):
        # 1) Sample latent z_i for each i
        mu_reg = compute_mu_reg(X, beta_g, groups)

        for i in range(n):
            if obs_mask[i]:
                wi = 1.0 / sigma2 + 1.0 / (s_obs_clean[i] ** 2)
                v_post = 1.0 / wi
                m_post = v_post * (mu_reg[i] / sigma2 + z_obs[i] / (s_obs_clean[i] ** 2))
                z[i] = np.random.normal(m_post, np.sqrt(v_post))
            else:
                z[i] = np.random.normal(mu_reg[i], np.sqrt(sigma2))

        # 2) Sample beta_g for each group g
        for g in range(J):
            idx = group_indices[g]
            if len(idx) == 0:
                continue

            X_g = X[idx, :]
            z_g = z[idx]

            XtX = X_g.T @ X_g
            tau2_safe = max(tau2, 1e-8)
            sigma2_safe = max(sigma2, 1e-8)
            prior_prec = np.eye(p) / tau2_safe

            precision = XtX / sigma2_safe + prior_prec
            precision = precision + jitter_prec * np.eye(p)

            try:
                V_g = np.linalg.inv(precision)
            except np.linalg.LinAlgError:
                precision = precision + 1e-4 * np.eye(p)
                V_g = np.linalg.inv(precision)

            rhs = X_g.T @ z_g / sigma2_safe + mu_beta / tau2_safe
            m_g = V_g @ rhs

            beta_g[g, :] = safe_sample_mvn(m_g, V_g)

        # 3) Sample mu_beta
        tau2_safe = max(tau2, 1e-8)
        scalar_prec = J / tau2_safe + 1.0 / (c0 ** 2)
        V_mu_scalar = 1.0 / scalar_prec
        sum_beta = beta_g.sum(axis=0)
        m_mu = V_mu_scalar * (sum_beta / tau2_safe)
        mu_beta = np.random.multivariate_normal(m_mu, V_mu_scalar * np.eye(p))

        # 4) Sample sigma2
        mu_reg = compute_mu_reg(X, beta_g, groups)
        resid = z - mu_reg
        a_post = a0 + n / 2.0
        b_post = b0 + 0.5 * np.nansum(resid ** 2)
        sigma2 = invgamma_sample(a_post, b_post)
        sigma2 = float(np.clip(sigma2, 1e-8, 1e8))

        # 5) Sample tau2
        diff = beta_g - mu_beta[None, :]
        a_tau_post = a_tau + (J * p) / 2.0
        b_tau_post = b_tau + 0.5 * np.nansum(diff ** 2)
        tau2 = invgamma_sample(a_tau_post, b_tau_post)
        tau2 = float(np.clip(tau2, 1e-8, 1e8))

        # Store samples
        if it > burn_in and ((it - burn_in) % thin == 0):
            beta_samples[keep_idx, :, :] = beta_g
            mu_beta_samples[keep_idx, :] = mu_beta
            sigma2_samples[keep_idx] = sigma2
            tau2_samples[keep_idx] = tau2
            keep_idx += 1

    # =====================================================
    # Summaries and plots
    # =====================================================

    summary = {}

    # Summary of mu_beta
    mu_mean = mu_beta_samples.mean(axis=0)
    mu_low = np.percentile(mu_beta_samples, 2.5, axis=0)
    mu_high = np.percentile(mu_beta_samples, 97.5, axis=0)
    df_mu = pd.DataFrame({
        "feature": [f for f in range(len(mu_mean))],
        "feature_name": feature_names,
        "mean": mu_mean,
        "ci_2p5": mu_low,
        "ci_97p5": mu_high
    })
    df_mu.to_csv(os.path.join(prop_dir, "mu_beta_summary.csv"), index=False)
    summary["mu_beta"] = df_mu

    # Summary of beta_g per group
    rows = []
    J = beta_samples.shape[1]
    p = beta_samples.shape[2]
    for g in range(J):
        beta_g_chain = beta_samples[:, g, :]
        b_mean = beta_g_chain.mean(axis=0)
        b_low = np.percentile(beta_g_chain, 2.5, axis=0)
        b_high = np.percentile(beta_g_chain, 97.5, axis=0)
        for j in range(p):
            rows.append({
                "group_index": g,
                "group_label": idx_to_class[g],
                "feature_index": j,
                "feature_name": feature_names[j],
                "mean": b_mean[j],
                "ci_2p5": b_low[j],
                "ci_97p5": b_high[j]
            })
    df_beta = pd.DataFrame(rows)
    df_beta.to_csv(os.path.join(prop_dir, "beta_group_summary.csv"), index=False)
    summary["beta_group"] = df_beta

    # Summary of sigma2 and tau2
    df_sigma_tau = pd.DataFrame({
        "sigma2_mean": [sigma2_samples.mean()],
        "sigma2_ci_2p5": [np.percentile(sigma2_samples, 2.5)],
        "sigma2_ci_97p5": [np.percentile(sigma2_samples, 97.5)],
        "tau2_mean": [tau2_samples.mean()],
        "tau2_ci_2p5": [np.percentile(tau2_samples, 2.5)],
        "tau2_ci_97p5": [np.percentile(tau2_samples, 97.5)],
    })
    df_sigma_tau.to_csv(os.path.join(prop_dir, "sigma2_tau2_summary.csv"), index=False)
    summary["sigma2_tau2"] = df_sigma_tau

    # -------- Trace plots for sigma2 and tau2 ----------
    plt.figure(figsize=(10, 4))
    plt.plot(sigma2_samples)
    plt.title(f"Trace of sigma^2 ({prop_name})")
    plt.xlabel("Saved iteration")
    plt.ylabel("sigma^2")
    plt.tight_layout()
    plt.savefig(os.path.join(prop_dir, "trace_sigma2.png"), dpi=300)
    plt.close()

    plt.figure(figsize=(10, 4))
    plt.plot(tau2_samples)
    plt.title(f"Trace of tau^2 ({prop_name})")
    plt.xlabel("Saved iteration")
    plt.ylabel("tau^2")
    plt.tight_layout()
    plt.savefig(os.path.join(prop_dir, "trace_tau2.png"), dpi=300)
    plt.close()

    # -------- Histogram of sigma2 and tau2 ----------
    plt.figure(figsize=(6, 4))
    plt.hist(sigma2_samples, bins=30, density=True)
    plt.title(f"Posterior of sigma^2 ({prop_name})")
    plt.xlabel("sigma^2")
    plt.tight_layout()
    plt.savefig(os.path.join(prop_dir, "hist_sigma2.png"), dpi=300)
    plt.close()

    plt.figure(figsize=(6, 4))
    plt.hist(tau2_samples, bins=30, density=True)
    plt.title(f"Posterior of tau^2 ({prop_name})")
    plt.xlabel("tau^2")
    plt.tight_layout()
    plt.savefig(os.path.join(prop_dir, "hist_tau2.png"), dpi=300)
    plt.close()

    # -------- Posterior predictive vs observed ----------
    mask_valid_obs = ~np.isnan(z_obs)
    n_valid = int(mask_valid_obs.sum())

    if n_valid >= 2:
        beta_last = beta_samples[-1, :, :]
        mu_reg_last = compute_mu_reg(X, beta_last, groups)
        z_pred_mean = mu_reg_last

        # Scatter
        plt.figure(figsize=(6, 6))
        obs_vals = z_obs[mask_valid_obs]
        pred_vals = z_pred_mean[mask_valid_obs]
        plt.scatter(obs_vals, pred_vals, alpha=0.6)
        min_val = np.nanmin(obs_vals)
        max_val = np.nanmax(obs_vals)
        plt.plot([min_val, max_val], [min_val, max_val], "k--", lw=1)
        plt.xlabel("Observed transformed value")
        plt.ylabel("Posterior mean (transformed)")
        plt.title(f"Observed vs Posterior Mean (transformed) - {prop_name}")
        plt.tight_layout()
        plt.savefig(os.path.join(prop_dir, "obs_vs_pred_transformed.png"), dpi=300)
        plt.close()

        # Residual histogram
        resid_last = obs_vals - pred_vals
        if np.all(np.isnan(resid_last)) or resid_last.size == 0:
            print(f"[{prop_name}] No finite residuals to plot residual histogram; skipping.")
        else:
            resid_last = resid_last[np.isfinite(resid_last)]
            if resid_last.size >= 1:
                plt.figure(figsize=(6, 4))
                plt.hist(resid_last, bins=30, density=True)
                plt.title(f"Residuals (transformed) - {prop_name}")
                plt.xlabel("z_obs - z_pred")
                plt.tight_layout()
                plt.savefig(os.path.join(prop_dir, "residuals_hist.png"), dpi=300)
                plt.close()
            else:
                print(f"[{prop_name}] Residuals all non-finite; skipping residual histogram.")
    else:
        print(f"[{prop_name}] No or too few valid transformed observations; skipping obs-vs-pred and residual plots.")

    # Print key summaries to console
    print(f"\nSummary for property: {prop_name}")
    print("Global mean coefficients (mu_beta) [first 10 features]:")
    print(df_mu.head(10))
    print("\nResidual variance and between-class variance:")
    print(df_sigma_tau)

    return {
        "beta_samples": beta_samples,
        "mu_beta_samples": mu_beta_samples,
        "sigma2_samples": sigma2_samples,
        "tau2_samples": tau2_samples,
        "summary": summary
    }


# =========================================================
# Main orchestration
# =========================================================

def main():
    ensure_dir(OUTPUT_DIR)

    # Load and prepare data
    df, X, feat_names_local, groups, idx_to_class, y_dict, s_dict = load_and_prepare_data(CSV_PATH)

    global feature_names
    feature_names = feat_names_local  # used inside MCMC function

    # Properties to analyse
    properties = ["YM", "YS", "UTS", "DAR", "HV"]

    all_results = {}

    for prop in properties:
        y = y_dict[prop]
        s = s_dict[prop]
        z_obs, s_trans = transform_property(y, s, prop)

        res = run_hierarchical_mcmc_property(
            X=X,
            groups=groups,
            z_obs=z_obs,
            s_obs=s_trans,
            prop_name=prop,
            idx_to_class=idx_to_class,
            n_iter=N_ITER,
            burn_in=BURN_IN,
            thin=THIN,
            output_dir=OUTPUT_DIR
        )
        all_results[prop] = res

    # Create a zip of all outputs
    zip_name = OUTPUT_DIR + ".zip"
    create_zip_from_directory(OUTPUT_DIR, zip_name)
    print(f"\nAll results saved in '{OUTPUT_DIR}' and zipped as '{zip_name}'.")


if __name__ == "__main__":
    main()



=== MCMC for property: YM ===
n = 283, p = 14, J (moe_class) = 5
Saving results to: results_dax_ti_bayes/property_YM


MCMC YM:   0%|          | 0/10000 [00:00<?, ?it/s]

[YM] No finite residuals to plot residual histogram; skipping.

Summary for property: YM
Global mean coefficients (mu_beta) [first 10 features]:
   feature      feature_name  mean  ci_2p5  ci_97p5
0        0         Intercept   NaN     NaN      NaN
1        1           moe_std   NaN     NaN      NaN
2        2            P0_std   NaN     NaN      NaN
3        3        P1_log_std   NaN     NaN      NaN
4        4      condition_PM   NaN     NaN      NaN
5        5   condition_PM-FC   NaN     NaN      NaN
6        6      condition_ST   NaN     NaN      NaN
7        7   condition_ST-AC   NaN     NaN      NaN
8        8  condition_ST-AC    NaN     NaN      NaN
9        9   condition_ST-FC   NaN     NaN      NaN

Residual variance and between-class variance:
   sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  \
0     0.006991       0.005821        0.008173   0.027989     0.020493   

   tau2_ci_97p5  
0      0.038844  

=== MCMC for property: YS ===
n = 283, p = 14, J (mo

MCMC YS:   0%|          | 0/10000 [00:00<?, ?it/s]

[YS] No finite residuals to plot residual histogram; skipping.

Summary for property: YS
Global mean coefficients (mu_beta) [first 10 features]:
   feature      feature_name  mean  ci_2p5  ci_97p5
0        0         Intercept   NaN     NaN      NaN
1        1           moe_std   NaN     NaN      NaN
2        2            P0_std   NaN     NaN      NaN
3        3        P1_log_std   NaN     NaN      NaN
4        4      condition_PM   NaN     NaN      NaN
5        5   condition_PM-FC   NaN     NaN      NaN
6        6      condition_ST   NaN     NaN      NaN
7        7   condition_ST-AC   NaN     NaN      NaN
8        8  condition_ST-AC    NaN     NaN      NaN
9        9   condition_ST-FC   NaN     NaN      NaN

Residual variance and between-class variance:
   sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  \
0     0.007002       0.006006        0.008138   0.027569     0.019621   

   tau2_ci_97p5  
0      0.038943  

=== MCMC for property: UTS ===
n = 283, p = 14, J (m

...

[UTS] No finite residuals to plot residual histogram; skipping.

Summary for property: UTS
Global mean coefficients (mu_beta) [first 10 features]:
   feature      feature_name  mean  ci_2p5  ci_97p5
0        0         Intercept   NaN     NaN      NaN
1        1           moe_std   NaN     NaN      NaN
2        2            P0_std   NaN     NaN      NaN
3        3        P1_log_std   NaN     NaN      NaN
4        4      condition_PM   NaN     NaN      NaN
5        5   condition_PM-FC   NaN     NaN      NaN
6        6      condition_ST   NaN     NaN      NaN
7        7   condition_ST-AC   NaN     NaN      NaN
8        8  condition_ST-AC    NaN     NaN      NaN
9        9   condition_ST-FC   NaN     NaN      NaN

Residual variance and between-class variance:
   sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  \
0     0.007023        0.00608        0.008293   0.027566     0.020292   

   tau2_ci_97p5  
0      0.037851  

=== MCMC for property: DAR ===
n = 283, p = 14, J 

MCMC DAR:   0%|          | 0/10000 [00:00<?, ?it/s]

[DAR] No finite residuals to plot residual histogram; skipping.

Summary for property: DAR
Global mean coefficients (mu_beta) [first 10 features]:
   feature      feature_name  mean  ci_2p5  ci_97p5
0        0         Intercept   NaN     NaN      NaN
1        1           moe_std   NaN     NaN      NaN
2        2            P0_std   NaN     NaN      NaN
3        3        P1_log_std   NaN     NaN      NaN
4        4      condition_PM   NaN     NaN      NaN
5        5   condition_PM-FC   NaN     NaN      NaN
6        6      condition_ST   NaN     NaN      NaN
7        7   condition_ST-AC   NaN     NaN      NaN
8        8  condition_ST-AC    NaN     NaN      NaN
9        9   condition_ST-FC   NaN     NaN      NaN

Residual variance and between-class variance:
   sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  \
0     0.007035       0.006019        0.008128   0.027736     0.019508   

   tau2_ci_97p5  
0      0.039677  

=== MCMC for property: HV ===
n = 283, p = 14, J (

MCMC HV:   0%|          | 0/10000 [00:00<?, ?it/s]

[HV] No finite residuals to plot residual histogram; skipping.

Summary for property: HV
Global mean coefficients (mu_beta) [first 10 features]:
   feature      feature_name  mean  ci_2p5  ci_97p5
0        0         Intercept   NaN     NaN      NaN
1        1           moe_std   NaN     NaN      NaN
2        2            P0_std   NaN     NaN      NaN
3        3        P1_log_std   NaN     NaN      NaN
4        4      condition_PM   NaN     NaN      NaN
5        5   condition_PM-FC   NaN     NaN      NaN
6        6      condition_ST   NaN     NaN      NaN
7        7   condition_ST-AC   NaN     NaN      NaN
8        8  condition_ST-AC    NaN     NaN      NaN
9        9   condition_ST-FC   NaN     NaN      NaN

Residual variance and between-class variance:
   sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  \
0     0.007015       0.005996        0.008311   0.027847     0.020209   

   tau2_ci_97p5  
0      0.038514  

All results saved in 'results_dax_ti_bayes' and zipp

##  Correction 2: Design Matrix NaNs and Posterior Outputs

### Core Issue (now fixed)

The MCMC was encountering **NaNs in the design matrix** due to missing values (e.g., `P0`), which caused the **latent mean \( \mu_{\text{reg}} \)** to become NaN for some alloys. This corrupted the downstream inference.

---

###  Fixes and Enhancements

- **Clean design matrix**:
  - All missing continuous covariates are **mean-imputed after standardisation**
  - Ensures **no NaNs** in \( X \)

- **Data-dependent priors**:
  - Empirical prior for global mean regression \( \mu_\beta \)  
  - Estimated via **weighted least squares** on log-transformed data

- **Back-transformation**:
  - Posterior predictive summaries are computed in **original units** (GPa / MPa / % / HV)

---

###  Script Overview

This complete, self-contained Python script:

1. **Loads** `dax-ti-static.csv`
2. **Builds** a clean design matrix \( X \) with:
   - Intercept  
   - Standardised and mean-imputed `moe`, `P0`, log(`P1`)  
   - One-hot dummies for `moe_class`, `condition`, `ph1`
3. **Uses** a **log-scale latent model** for each property: `YM`, `YS`, `UTS`, `DAR`, `HV`
4. **Implements** hierarchical Gaussian MCMC with:
   - Manual updates (no MCMC libraries)  
   - `tqdm` progress bar for sampling

---

### For Each Property

- Saves **\( \mu_\beta \)** summaries to CSV  
- Saves **\( \sigma^2 \)** and **\( \tau^2 \)** summaries to CSV  
- Generates **residual histograms** (log-scale)  
- Computes **posterior predictive distributions** in original units  
- Saves **per-alloy predictive summaries** to CSV  
- Plots:
  - Observed vs predicted (original units)  
  - Histogram overlays  
- Prints **concise console summaries**

---

###  Final Output

- **Zips the full results folder** for convenient download

---

This version is robust, interpretable, and ready for reproducible inference and publication.

## 🛠️ Correction 3: NaNs and Broadcasting Error (Now Fixed)

###  Root Cause

The issue stemmed from:
- NaNs creeping into the **design matrix**  
- A brittle `compute_mu_reg` implementation using `np.einsum`, which caused **shape mismatches**

---

###  Fixes Implemented

- `compute_mu_reg` is now a **simple, explicit loop over groups**  
  - No `einsum`, no broadcasting errors  
- **Continuous covariates** are:
  - Standardized  
  - Mean-imputed **on the standardized scale**  
  - Result: design matrix \( X \) has **no NaNs**
- A **data-dependent prior** for \( \mu_\beta \) is used  
  - Estimated via **weighted least squares** on \( z_{\text{obs}} \), accounting for reported errors

---

###  For Each Property (`YM`, `YS`, `UTS`, `DAR`, `HV`)

- Runs **hierarchical MCMC**:
  - `N_ITER = 10000`, `BURN_IN = 4000`, `THIN = 10`  
  - Operates on **log scale** for latent property
- Computes **posterior predictive draws** in original units:
  - GPa / MPa / % / HV

---

###  Outputs Saved

- `mu_beta_summary_<prop>.csv`  
- `variance_summary_<prop>.csv`  
- `posterior_predictive_<prop>.csv`  
- Console summary includes **first few rows** of each predictive CSV  
- Multiple plots:
  - Trace plots  
  - Posterior histograms  
  - Observed vs predicted (original units)  
  - Histogram overlays

---

###  Final Step

- All results are saved under `results_dax_ti_bayes/`  
- Zipped into `dax_ti_bayes_results.zip` for download

---

###  To Run

Place `dax-ti-static.csv` in the working directory and run this script in **Colab**. Everything is self-contained and ready to go.

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Bayesian hierarchical model for Ti-alloy mechanical properties
using data-dependent empirical priors and manual MCMC.

Dataset: dax-ti-static.csv (Salvador et al., Scientific Data, 2022)
"""

import os
from pathlib import Path
import zipfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm


# ============================================================
# Utility helpers
# ============================================================

def ensure_dir(path: str):
    """Create directory if it does not exist."""
    Path(path).mkdir(parents=True, exist_ok=True)


def zip_directory(root_dir: str, zip_path: str):
    """Zip a directory recursively for easy download."""
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for foldername, _, filenames in os.walk(root_dir):
            for fn in filenames:
                full_path = os.path.join(foldername, fn)
                arcname = os.path.relpath(full_path, root_dir)
                zf.write(full_path, arcname)


def standardize_series(s: pd.Series):
    """
    Standardize a pandas Series: (x - mean)/sd, robust to all-NaN or zero variance.
    Returns (standardized_series, mean, sd).
    """
    mu = s.mean()
    sd = s.std(ddof=1)
    if not np.isfinite(sd) or sd == 0:
        sd = 1.0
    return (s - mu) / sd, mu, sd


# ============================================================
# Data loading and preparation
# ============================================================

def load_and_prepare_data(csv_path: str):
    """
    Load Salvador et al. Ti-alloy dataset and construct design matrix X,
    group labels, and property/error vectors.

    - Continuous covariates: moe, P0, log(1+P1)
    - Categorical covariates: condition, ph1 (one-hot, drop_first=True)
    - Missing continuous covariates are standardized then mean-imputed (0 on std scale).
    - moe_class used as grouping factor.
    """
    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]

    # Keep only entries with defined Mo[eq] class
    df = df[~df["moe_class"].isna()].copy()

    # Safe log transform for P1 (grain size etc.), imputing median for missing
    df["P1_log"] = np.log1p(df["P1"].fillna(df["P1"].median()))

    # Continuous covariates
    covariates_cont = ["moe", "P0", "P1_log"]
    for c in covariates_cont:
        std, mu, sd = standardize_series(df[c])
        # mean-impute missing after standardization
        df[c + "_std"] = std.fillna(0.0)

    # Categorical covariates
    cat_cols = ["condition", "ph1"]
    df[cat_cols] = df[cat_cols].fillna("missing")
    df_dummies = pd.get_dummies(df[cat_cols],
                                prefix=cat_cols,
                                drop_first=True)

    # Design matrix X with intercept
    X_df = pd.concat(
        [
            pd.Series(1.0, index=df.index, name="Intercept"),
            df[[c + "_std" for c in covariates_cont]],
            df_dummies
        ],
        axis=1
    )
    feature_names = list(X_df.columns)
    X = X_df.to_numpy(dtype=float)

    # Grouping by Mo[eq] class
    moe_classes = df["moe_class"].astype(str)
    class_levels = sorted(moe_classes.unique())
    class_to_idx = {c: i for i, c in enumerate(class_levels)}
    idx_to_class = {i: c for c, i in class_to_idx.items()}
    groups = moe_classes.map(class_to_idx).to_numpy(dtype=int)

    # Mechanical properties and their reported errors
    prop_cols = {
        "YM":  ("YM", "YM_err"),    # Young's modulus (GPa)
        "YS":  ("YS", "YS_err"),    # Yield strength (MPa)
        "UTS": ("UTS", "UTS_err"),  # Ultimate strength (MPa)
        "DAR": ("DAR", "DAR_err"),  # Deformation at rupture (%)
        "HV":  ("HV", "HV_err"),    # Vickers hardness
    }

    y_dict, s_dict = {}, {}
    rel_default = 0.05  # 5% relative error as default

    for prop, (v_col, e_col) in prop_cols.items():
        y = df[v_col].to_numpy(dtype=float)
        s = df[e_col].to_numpy(dtype=float)

        # Clean error: if missing or non-positive, set to 5% of |y|
        s_clean = s.copy()
        mask_bad = np.isnan(s_clean) | (s_clean <= 0)
        s_clean[mask_bad] = rel_default * np.abs(y[mask_bad])

        y_dict[prop] = y
        s_dict[prop] = s_clean

    return df, X, feature_names, groups, idx_to_class, y_dict, s_dict


# ============================================================
# Transformations between original and latent scales
# ============================================================

def transform_property(y: np.ndarray, s: np.ndarray, prop_name: str):
    """
    Map property to latent scale (log for positive-valued properties) and
    propagate measurement errors.

    For YM, YS, UTS, DAR, HV:
        z = log(y),  s_z ≈ s / y   (relative error approximation)
    """
    y = y.copy()
    s = s.copy()

    if prop_name in ["YM", "YS", "UTS", "HV", "DAR"]:
        invalid = (y <= 0) | np.isnan(y)
        z = np.full_like(y, np.nan, dtype=float)
        z[~invalid] = np.log(y[~invalid])

        s_trans = np.full_like(s, np.nan, dtype=float)
        s_trans[~invalid] = np.abs(s[~invalid] / y[~invalid])
    else:
        raise ValueError(f"Unknown property {prop_name}")

    return z, s_trans


def inverse_transform(z_draws: np.ndarray, prop_name: str):
    """
    Inverse transform from latent scale to original units.

    For log-transformed properties, simply exp().
    """
    if prop_name in ["YM", "YS", "UTS", "HV", "DAR"]:
        return np.exp(z_draws)
    else:
        raise ValueError(f"Unknown property {prop_name}")


# ============================================================
# Basic distributions and linear algebra helpers
# ============================================================

def invgamma_sample(a: float, b: float) -> float:
    """
    Sample from Inverse-Gamma(a, b) with defensive checks.
    p(x) ∝ x^{-(a+1)} exp(-b/x)
    """
    a = float(a)
    b = float(b)
    if not np.isfinite(a) or a <= 0:
        a = 1.0
    if not np.isfinite(b) or b <= 0:
        b = 1.0
    g = np.random.gamma(shape=a, scale=1.0 / b)
    if g <= 0 or not np.isfinite(g):
        g = 1.0
    return 1.0 / g


def safe_sample_mvn(mean: np.ndarray,
                    cov: np.ndarray,
                    jitter: float = 1e-8,
                    max_tries: int = 5) -> np.ndarray:
    """
    Sample from multivariate normal N(mean, cov) using eigen-decomposition,
    with increasing jitter to stabilise near-singular matrices.
    """
    p = cov.shape[0]
    cov = 0.5 * (cov + cov.T)  # enforce symmetry

    for _ in range(max_tries):
        try:
            w, v = np.linalg.eigh(cov)
            w = np.clip(w, jitter, None)
            L = v @ np.diag(np.sqrt(w))
            z = np.random.randn(p)
            return mean + L @ z
        except np.linalg.LinAlgError:
            cov = cov + jitter * np.eye(p)
            jitter *= 10

    # Fallback: diagonal approximation
    diag = np.clip(np.diag(cov), jitter, None)
    z = np.random.randn(p)
    return mean + np.sqrt(diag) * z


def compute_mu_reg(X: np.ndarray,
                   beta_g: np.ndarray,
                   groups: np.ndarray) -> np.ndarray:
    """
    Compute regression mean μ_reg_i = x_i' β_{g[i]} for all i.

    X:       (n x p)
    beta_g:  (J x p)
    groups:  (n,) integers in {0,...,J-1}
    """
    n, p = X.shape
    J, p2 = beta_g.shape
    if p2 != p:
        raise ValueError(
            f"Dimension mismatch in compute_mu_reg: X is (n={n}, p={p}), "
            f"beta_g is (J={J}, p2={p2})."
        )

    mu = np.zeros(n, dtype=float)
    for g in range(J):
        idx = (groups == g)
        if not np.any(idx):
            continue
        mu[idx] = X[idx, :] @ beta_g[g, :]
    return mu


# ============================================================
# Empirical (data-dependent) prior for μ_β
# ============================================================

def empirical_prior_mu_beta(z_obs: np.ndarray,
                            s_obs_clean: np.ndarray,
                            X: np.ndarray,
                            ridge: float = 1e-4) -> np.ndarray:
    """
    Compute a data-dependent prior mean μ_0 for μ_β via weighted least squares:

        z_obs ≈ X β_OLS + ε,   ε ∼ N(0, s_obs^2)

    Returns β_OLS as empirical prior centre μ_0.
    """
    mask = ~np.isnan(z_obs)
    if mask.sum() < X.shape[1]:
        # Too few observations to estimate reliably
        return np.zeros(X.shape[1])

    Xv = X[mask, :]
    zv = z_obs[mask]
    w = 1.0 / (s_obs_clean[mask] ** 2 + 1e-8)

    # Weighted least squares: (X'WX + λI)^(-1) X'Wz
    XtWX = Xv.T @ (Xv * w[:, None])
    XtWz = Xv.T @ (w * zv)
    XtWX_reg = XtWX + ridge * np.eye(X.shape[1])

    beta_hat = np.linalg.solve(XtWX_reg, XtWz)
    return beta_hat


# ============================================================
# Posterior predictive in original units
# ============================================================

def posterior_predictive(
    X: np.ndarray,
    groups: np.ndarray,
    z_obs: np.ndarray,
    s_obs_clean: np.ndarray,
    y_orig: np.ndarray,
    prop_name: str,
    idx_to_class: dict,
    beta_samples: np.ndarray,
    sigma2_samples: np.ndarray,
    output_dir: str,
    df_ids: pd.DataFrame = None
):
    """
    Compute posterior predictive distribution for y in original units and
    save summaries + plots.

    For each MCMC draw m and alloy i:
        μ_{mi} = x_i' β_{g[i]}^{(m)}
        z_rep_{mi} ∼ N( μ_{mi}, σ2^{(m)} + s_i^2 )   (latent log scale)
        y_rep_{mi} = exp(z_rep_{mi})                 (original units)
    """
    ensure_dir(output_dir)

    n = X.shape[0]
    M = beta_samples.shape[0]
    J = beta_samples.shape[1]

    # Precompute per-group indices
    group_indices = {g: np.where(groups == g)[0] for g in range(J)}

    # 1) Posterior draws of μ_i (M x n)
    mu_mat = np.zeros((M, n), dtype=float)
    for m in range(M):
        Bm = beta_samples[m, :, :]  # (J x p)
        mu = np.zeros(n, dtype=float)
        for g, idx in group_indices.items():
            if len(idx) == 0:
                continue
            mu[idx] = X[idx, :] @ Bm[g, :]
        mu_mat[m, :] = mu

    # 2) Sample latent z_rep and back-transform to y_rep
    var_mat = sigma2_samples[:, None] + (s_obs_clean[None, :] ** 2)
    z_rep = np.random.normal(loc=mu_mat, scale=np.sqrt(var_mat))
    y_rep = inverse_transform(z_rep, prop_name)

    # 3) Summaries per alloy
    y_mean = np.nanmean(y_rep, axis=0)
    y_median = np.nanmedian(y_rep, axis=0)
    y_low = np.nanpercentile(y_rep, 2.5, axis=0)
    y_high = np.nanpercentile(y_rep, 97.5, axis=0)

    if df_ids is not None and "id" in df_ids.columns:
        id_vals = df_ids["id"].values
    else:
        id_vals = np.arange(n)

    group_labels = np.array([idx_to_class[g] for g in groups])

    df_pred = pd.DataFrame({
        "id": id_vals,
        "group": group_labels,
        f"{prop_name}_obs": y_orig,
        f"{prop_name}_pred_mean": y_mean,
        f"{prop_name}_pred_median": y_median,
        f"{prop_name}_pred_ci2p5": y_low,
        f"{prop_name}_pred_ci97p5": y_high,
    })

    csv_path = os.path.join(output_dir,
                            f"posterior_predictive_{prop_name}.csv")
    df_pred.to_csv(csv_path, index=False)

    # 4) Simple predictive metrics on valid observations
    mask_valid = ~np.isnan(y_orig) & (y_orig > 0)
    if mask_valid.sum() > 0:
        y_obs_valid = y_orig[mask_valid]
        y_pred_valid = y_mean[mask_valid]

        rmse = float(np.sqrt(np.mean((y_obs_valid - y_pred_valid) ** 2)))
        mae = float(np.mean(np.abs(y_obs_valid - y_pred_valid)))
        ss_res = np.sum((y_obs_valid - y_pred_valid) ** 2)
        ss_tot = np.sum((y_obs_valid - y_obs_valid.mean()) ** 2)
        r2 = float(1 - ss_res / ss_tot) if ss_tot > 0 else np.nan
    else:
        rmse = mae = r2 = np.nan

    # 5) Plots: observed vs predicted, and histograms
    if mask_valid.sum() > 0:
        plt.figure()
        plt.scatter(y_obs_valid, y_pred_valid, alpha=0.6)
        min_val = min(y_obs_valid.min(), y_pred_valid.min())
        max_val = max(y_obs_valid.max(), y_pred_valid.max())
        plt.plot([min_val, max_val], [min_val, max_val],
                 "k--", linewidth=1)
        plt.xlabel(f"Observed {prop_name} (original units)")
        plt.ylabel(f"Predicted mean {prop_name}")
        plt.title(f"{prop_name}: Observed vs Predicted\n"
                  f"RMSE={rmse:.3g}, R²={r2:.3g}")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir,
                                 f"{prop_name}_obs_vs_pred.png"), dpi=300)
        plt.close()

        plt.figure()
        plt.hist(y_mean[mask_valid], bins=30, alpha=0.7,
                 label="Predicted mean")
        plt.hist(y_orig[mask_valid], bins=30, alpha=0.5,
                 label="Observed")
        plt.xlabel(f"{prop_name} (original units)")
        plt.ylabel("Count")
        plt.legend()
        plt.title(f"{prop_name}: Observed vs Posterior Predictive Mean")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir,
                                 f"{prop_name}_hist_obs_vs_pred.png"),
                    dpi=300)
        plt.close()

    metrics = {"rmse": rmse, "mae": mae, "r2": r2, "csv_path": csv_path}
    return df_pred, metrics


# ============================================================
# Hierarchical MCMC for a single property
# ============================================================

def run_hierarchical_mcmc_property(
    df: pd.DataFrame,
    X: np.ndarray,
    feature_names: list,
    groups: np.ndarray,
    idx_to_class: dict,
    y: np.ndarray,
    s: np.ndarray,
    prop_name: str,
    n_iter: int = 10000,
    burn_in: int = 4000,
    thin: int = 10,
    output_dir: str = "results_dax_ti_bayes",
    seed: int = 123
):
    """
    Run hierarchical Gaussian regression on latent log-scale property:

        z_i | β_{g[i]}, σ²  ∼ N(x_i' β_{g[i]}, σ²)
        β_g | μ_β, τ²       ∼ N_p(μ_β, τ² I_p)
        μ_β                 ∼ N_p(μ_0, c0² I_p)   (data-dependent μ_0)
        σ², τ²              ∼ Inverse-Gamma

    Measurement error enters through z_obs and its error s_obs_clean.
    """
    np.random.seed(seed)
    ensure_dir(output_dir)
    prop_dir = os.path.join(output_dir, f"property_{prop_name}")
    ensure_dir(prop_dir)

    # Transform to latent scale with measurement errors
    z_obs, s_trans = transform_property(y, s, prop_name)
    n, p = X.shape
    J = len(np.unique(groups))

    print(f"=== MCMC for property: {prop_name} ===")
    print(f"n = {n}, p = {p}, J (moe_class) = {J}")
    print(f"Saving results to: {prop_dir}")

    obs_mask = ~np.isnan(z_obs)

    # Measurement std dev on log scale; huge for missing
    s_obs_clean = s_trans.copy()
    s_obs_clean[np.isnan(s_obs_clean)] = 1e6

    # Data-dependent prior centre for μ_β
    mu0 = empirical_prior_mu_beta(z_obs, s_obs_clean, X)

    # Hyperparameters (can be tuned if needed)
    a0, b0 = 2.0, 0.5       # prior for σ² ~ IG(a0, b0)
    a_tau, b_tau = 2.0, 0.5  # prior for τ² ~ IG(a_tau, b_tau)
    c0 = 5.0                # prior sd for μ_β around μ0
    jitter_prec = 1e-6

    # Initial states
    z = np.zeros(n)
    z[obs_mask] = z_obs[obs_mask]
    z[~obs_mask] = 0.0

    beta_g = np.tile(mu0, (J, 1))  # start at empirical prior centre
    mu_beta = mu0.copy()
    sigma2 = 0.05
    tau2 = 0.1

    group_indices = {g: np.where(groups == g)[0] for g in range(J)}

    # Storage
    n_keep = (n_iter - burn_in) // thin
    beta_samples = np.zeros((n_keep, J, p))
    mu_beta_samples = np.zeros((n_keep, p))
    sigma2_samples = np.zeros(n_keep)
    tau2_samples = np.zeros(n_keep)
    keep_idx = 0

    # MCMC loop
    for it in tqdm(range(1, n_iter + 1),
                   desc=f"MCMC {prop_name}",
                   leave=True):

        # 1) Sample latent z_i
        mu_reg = compute_mu_reg(X, beta_g, groups)
        for i in range(n):
            if obs_mask[i]:
                wi = 1.0 / sigma2 + 1.0 / (s_obs_clean[i] ** 2)
                v_post = 1.0 / wi
                m_post = v_post * (mu_reg[i] / sigma2 +
                                   z_obs[i] / (s_obs_clean[i] ** 2))
                z[i] = np.random.normal(m_post, np.sqrt(v_post))
            else:
                z[i] = np.random.normal(mu_reg[i], np.sqrt(sigma2))

        # 2) Sample β_g for each Mo[eq] class g
        for g in range(J):
            idx = group_indices[g]
            if len(idx) == 0:
                continue
            X_g = X[idx, :]
            z_g = z[idx]

            XtX = X_g.T @ X_g
            tau2_safe = max(tau2, 1e-8)
            sigma2_safe = max(sigma2, 1e-8)
            prior_prec = np.eye(p) / tau2_safe
            precision = XtX / sigma2_safe + prior_prec
            precision = precision + jitter_prec * np.eye(p)

            try:
                V_g = np.linalg.inv(precision)
            except np.linalg.LinAlgError:
                precision = precision + 1e-4 * np.eye(p)
                V_g = np.linalg.inv(precision)

            rhs = X_g.T @ z_g / sigma2_safe + mu_beta / tau2_safe
            m_g = V_g @ rhs
            beta_g[g, :] = safe_sample_mvn(m_g, V_g)

        # 3) Sample μ_β with prior N(μ0, c0² I)
        tau2_safe = max(tau2, 1e-8)
        scalar_prec = J / tau2_safe + 1.0 / (c0 ** 2)
        V_mu_scalar = 1.0 / scalar_prec
        beta_bar = beta_g.mean(axis=0)
        m_mu = V_mu_scalar * (J * beta_bar / tau2_safe + mu0 / (c0 ** 2))
        mu_beta = np.random.multivariate_normal(m_mu,
                                                V_mu_scalar * np.eye(p))

        # 4) Sample σ²
        mu_reg = compute_mu_reg(X, beta_g, groups)
        resid = z - mu_reg
        a_post = a0 + n / 2.0
        b_post = b0 + 0.5 * np.nansum(resid ** 2)
        sigma2 = invgamma_sample(a_post, b_post)
        sigma2 = float(np.clip(sigma2, 1e-8, 1e3))

        # 5) Sample τ²
        diff = beta_g - mu_beta[None, :]
        a_tau_post = a_tau + (J * p) / 2.0
        b_tau_post = b_tau + 0.5 * np.nansum(diff ** 2)
        tau2 = invgamma_sample(a_tau_post, b_tau_post)
        tau2 = float(np.clip(tau2, 1e-8, 1e3))

        # 6) Store draws
        if it > burn_in and ((it - burn_in) % thin == 0):
            beta_samples[keep_idx, :, :] = beta_g
            mu_beta_samples[keep_idx, :] = mu_beta
            sigma2_samples[keep_idx] = sigma2
            tau2_samples[keep_idx] = tau2
            keep_idx += 1

    # --------------------------------------------------------
    # Summaries for μ_β
    # --------------------------------------------------------
    mu_mean = mu_beta_samples.mean(axis=0)
    ci_low = np.percentile(mu_beta_samples, 2.5, axis=0)
    ci_high = np.percentile(mu_beta_samples, 97.5, axis=0)

    summary_mu = pd.DataFrame({
        "feature": np.arange(p),
        "feature_name": feature_names,
        "mean": mu_mean,
        "ci_2p5": ci_low,
        "ci_97p5": ci_high,
    })
    summary_mu_path = os.path.join(prop_dir,
                                   f"mu_beta_summary_{prop_name}.csv")
    summary_mu.to_csv(summary_mu_path, index=False)

    # --------------------------------------------------------
    # Summaries for σ² and τ²
    # --------------------------------------------------------
    summary_var = pd.DataFrame({
        "sigma2_mean": [sigma2_samples.mean()],
        "sigma2_ci_2p5": [np.percentile(sigma2_samples, 2.5)],
        "sigma2_ci_97p5": [np.percentile(sigma2_samples, 97.5)],
        "tau2_mean": [tau2_samples.mean()],
        "tau2_ci_2p5": [np.percentile(tau2_samples, 2.5)],
        "tau2_ci_97p5": [np.percentile(tau2_samples, 97.5)],
    })
    summary_var_path = os.path.join(prop_dir,
                                    f"variance_summary_{prop_name}.csv")
    summary_var.to_csv(summary_var_path, index=False)

    # --------------------------------------------------------
    # Residual histogram on latent (log) scale, last state
    # --------------------------------------------------------
    mu_reg_post = compute_mu_reg(X, beta_g, groups)
    resid_obs = z_obs[obs_mask] - mu_reg_post[obs_mask]
    if np.isfinite(resid_obs).any():
        plt.figure()
        plt.hist(resid_obs, bins=30, alpha=0.7)
        plt.xlabel(f"Residuals in log-{prop_name}")
        plt.ylabel("Count")
        plt.title(f"{prop_name}: Residuals (last state)")
        plt.tight_layout()
        plt.savefig(os.path.join(prop_dir,
                                 f"{prop_name}_residual_hist.png"),
                    dpi=300)
        plt.show()
        plt.close()
    else:
        print(f"[{prop_name}] No finite residuals to plot residual histogram.")

    # --------------------------------------------------------
    # Posterior predictive in original units
    # --------------------------------------------------------
    df_pred, metrics = posterior_predictive(
        X=X,
        groups=groups,
        z_obs=z_obs,
        s_obs_clean=s_obs_clean,
        y_orig=y,
        prop_name=prop_name,
        idx_to_class=idx_to_class,
        beta_samples=beta_samples,
        sigma2_samples=sigma2_samples,
        output_dir=prop_dir,
        df_ids=df,
    )

    # Console summaries
    print(f"\nSummary for property: {prop_name}")
    print("Global mean coefficients (mu_beta) [first 10 features]:")
    print(summary_mu.head(10).to_string(index=False))
    print("\nResidual variance and between-class variance:")
    print(summary_var.to_string(index=False))
    print("\nPosterior predictive metrics (original units):")
    print(f"RMSE = {metrics['rmse']:.3g}, "
          f"MAE = {metrics['mae']:.3g}, "
          f"R² = {metrics['r2']:.3g}")
    print(f"Posterior predictive CSV saved to: {metrics['csv_path']}")
    print("\nFirst 5 posterior predictive rows (original units):")
    print(df_pred.head(5).to_string(index=False))

    return {
        "beta_samples": beta_samples,
        "mu_beta_samples": mu_beta_samples,
        "sigma2_samples": sigma2_samples,
        "tau2_samples": tau2_samples,
        "summary_mu": summary_mu,
        "summary_var": summary_var,
        "metrics": metrics,
        "df_pred": df_pred,
        "prop_dir": prop_dir,
    }


# ============================================================
# Main driver: run for all properties and zip results
# ============================================================

def main():
    # Adjust path if needed
    csv_path = "dax-ti-static.csv"

    # Global MCMC settings (can be tuned)
    N_ITER = 10000
    BURN_IN = 4000
    THIN = 10
    OUTPUT_DIR = "results_dax_ti_bayes"

    df, X, feature_names, groups, idx_to_class, y_dict, s_dict = \
        load_and_prepare_data(csv_path)

    properties = ["YM", "YS", "UTS", "DAR", "HV"]
    results = {}

    for j, prop in enumerate(properties):
        res = run_hierarchical_mcmc_property(
            df=df,
            X=X,
            feature_names=feature_names,
            groups=groups,
            idx_to_class=idx_to_class,
            y=y_dict[prop],
            s=s_dict[prop],
            prop_name=prop,
            n_iter=N_ITER,
            burn_in=BURN_IN,
            thin=THIN,
            output_dir=OUTPUT_DIR,
            seed=123 + j,
        )
        results[prop] = res

    # Zip all results for easy download
    zip_path = os.path.join(OUTPUT_DIR, "dax_ti_bayes_results.zip")
    zip_directory(OUTPUT_DIR, zip_path)
    print(f"\nAll results zipped to: {zip_path}")


if __name__ == "__main__":
    main()


=== MCMC for property: YM ===
n = 283, p = 14, J (moe_class) = 5
Saving results to: results_dax_ti_bayes/property_YM


MCMC YM:   0%|          | 0/10000 [00:00<?, ?it/s]

/tmp/ipython-input-4079125084.py:169: RuntimeWarning: overflow encountered in exp
  return np.exp(z_draws)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)



Summary for property: YM
Global mean coefficients (mu_beta) [first 10 features]:
 feature     feature_name      mean    ci_2p5  ci_97p5
       0        Intercept  4.591240  4.045176 5.190291
       1          moe_std -0.055124 -0.359177 0.206962
       2           P0_std -0.035531 -0.328143 0.290113
       3       P1_log_std  0.018833 -0.243550 0.296367
       4     condition_PM  0.037692 -0.517671 0.614359
       5  condition_PM-FC  0.024639 -0.378015 0.442961
       6     condition_ST -0.129180 -0.686309 0.451132
       7  condition_ST-AC -0.081630 -0.393941 0.238580
       8 condition_ST-AC  -0.039397 -0.565635 0.466715
       9  condition_ST-FC -0.015126 -0.525125 0.523694

Residual variance and between-class variance:
 sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  tau2_ci_97p5
    0.038709       0.031862        0.046871   0.066395     0.035704      0.117915

Posterior predictive metrics (original units):
RMSE = 15.4, MAE = 10.7, R² = 0.495
Posterior predicti

MCMC YS:   0%|          | 0/10000 [00:00<?, ?it/s]

...

/tmp/ipython-input-4079125084.py:169: RuntimeWarning: overflow encountered in exp
  return np.exp(z_draws)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)



Summary for property: DAR
Global mean coefficients (mu_beta) [first 10 features]:
 feature     feature_name      mean    ci_2p5  ci_97p5
       0        Intercept  0.287676 -1.190453 1.654588
       1          moe_std -0.007736 -0.621299 0.641570
       2           P0_std -0.057921 -0.716784 0.521880
       3       P1_log_std -0.242031 -0.787124 0.269535
       4     condition_PM -0.390771 -1.841250 0.995554
       5  condition_PM-FC  0.135790 -0.874252 1.061185
       6     condition_ST  2.948280  1.566339 4.585451
       7  condition_ST-AC  0.809392 -0.054080 1.685373
       8 condition_ST-AC   1.085170 -0.479978 2.407032
       9  condition_ST-FC  0.935513 -0.094896 2.060632

Residual variance and between-class variance:
 sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  tau2_ci_97p5
    0.453017       0.360283        0.556162   0.209242     0.086556      0.464201

Posterior predictive metrics (original units):
RMSE = 13.2, MAE = 9.51, R² = 0.284
Posterior predict

MCMC HV:   0%|          | 0/10000 [00:00<?, ?it/s]

/tmp/ipython-input-4079125084.py:169: RuntimeWarning: overflow encountered in exp
  return np.exp(z_draws)
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)



Summary for property: HV
Global mean coefficients (mu_beta) [first 10 features]:
 feature     feature_name      mean    ci_2p5  ci_97p5
       0        Intercept  5.770792  5.237582 6.282646
       1          moe_std  0.024708 -0.306857 0.354348
       2           P0_std  0.038743 -0.311943 0.395260
       3       P1_log_std  0.162258 -0.173270 0.492509
       4     condition_PM  0.023954 -0.596869 0.581514
       5  condition_PM-FC -1.651698 -5.081276 2.104631
       6     condition_ST -1.503042 -6.895717 3.672134
       7  condition_ST-AC -0.113048 -0.627840 0.391474
       8 condition_ST-AC  -2.684339 -7.921424 2.367405
       9  condition_ST-FC -0.095495 -0.534276 0.333297

Residual variance and between-class variance:
 sigma2_mean  sigma2_ci_2p5  sigma2_ci_97p5  tau2_mean  tau2_ci_2p5  tau2_ci_97p5
    0.060418       0.047968        0.076648   0.083851     0.041175      0.163871

Posterior predictive metrics (original units):
RMSE = 65.1, MAE = 47.2, R² = 0.529
Posterior predicti